<h1><b>Fast, Flexible, Easy and Intuitive: How to Speed Up Your pandas Projects</b></h1>
<p>In this notebook, I will be covering an important concept that users must know once they get acquainted with Pandas: the <code>SettingWithCopyWarning</code> issue. As a beginner, I would often disregard this sign as the outcome of the code would not necessarily change (apparently). Now, there are <b>shallow copies</b> and <b>deep copies</b>, notions that are related to the aforementioned warning. I'll discuss the difference between these concepts as well as assessing the impact of their use in data analysis.</p>
<p>By the end of this notebook, I hope to have covered the following topics:</p>
<ul>
    <li>The definition of <b>views</b> and <b>copies</b> in NumPy and Pandas</li>
    <li>How to work with views and copies in these libraries</li>
    <li>Why <code>SettingWithCopyWarning</code> happens in Pandas</li>
    <li>How to avoid getting a <code>SettingWithCopyWarning</code> in Pandas</li>
</ul>

In [18]:
import pandas as pd
import os
import time
import functools
import gc
import itertools
import sys
from timeit import default_timer as _timer

In [19]:
path = os.getcwd()
print(os.getcwd())

c:\Users\Felipe\python_work\notebooks\py_pandas\8_fast_flexible_pandas


In [20]:
data_repo = f"{path}/data/"
data_in = f"{data_repo}raw/"

In [21]:
df = pd.read_csv(f"{data_in}demand_profile.csv")

In [22]:
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [23]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [24]:
# df["date_time"] = pd.to_datetime(df["date_time"])
df["date_time"].dtype

dtype('O')

In [25]:
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [26]:
# def timeit(func):
#     def wrapper(*args, **kwargs):
#         start_time = time.time()
#         result = func(*args, **kwargs)
#         end_time = time.time()
#         execution_time = end_time - start_time
#         print(f"Execution time: {execution_time} seconds")
#         return result
#     return wrapper


def timeit(_func=None, *, repeat=3, number=1000, file=sys.stdout):
    """Decorator: prints time from best of `repeat` trials.
    Mimics `timeit.repeat()`, but avg. time is printed.
    Returns function result and prints time.

    You can decorate with or without parentheses, as in
    Python's @dataclass class decorator,

    kwargs are passed to `print()`.

    >>> @timeit
    ... def f():
    ...     return "-".join(str(n) for n in range(100))
    ...
    >>> @timeit(number=100000)
    ... def g():
    ...     return "-".join(str(n) for n in range(10))
    ...
    """

    _repeat = functools.partial(itertools.repeat, None)

    def wrap(func):
        @functools.wraps(func)
        def _timeit(*args, **kwargs):
            # Temporarily turn off garbage collection during the timing.
            # Makes independent timings more comparable.
            # If it was originally enabled, switch it back on afterwards.
            gcold = gc.isenabled()
            gc.disable()

            try:
                # Outer loop - the number of repeats.
                trials = []
                for _ in _repeat(repeat):
                    # Inner loop - the number of calls within each repeat.
                    total = 0
                    for _ in _repeat(number):
                        start = _timer()
                        result = func(*args, **kwargs)
                        end = _timer()
                        total += end - start
                    trials.append(total)
                
                # We want the *average_time* from the *best* trial
                # For more on this methodology, see the docs for
                # Python's `timeit` module.
                # 
                # "In a typical casem the lowest value gives a lower bound
                # for how fast your machine can run the given code snippet;
                # higher values in the result vector are typically not
                # caused by variability in Python's speed, but by other
                # processes interfering with your timing accuracy."
                best = min(trials) / number
                print(
                    "Best of {} trials with {} function"
                    " calls per trial:".format(repeat, number)
                )

                print(
                    "Function `{}` ran in average"
                    " of {:0.3f} seconds".format(func.__name__, best),
                    end="\n\n",
                    file=file,
                )
            finally:
                if gcold:
                    gc.enable()
            # Result is returned only once
            return result
        
        return _timeit
    
    # Syntax trick from Python @dataclass
    if _func is None:
        return wrap
    else:
        return wrap(_func)


@timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

# Read it again so that we have `object` dtype to start
df['date_time'] = convert(df, 'date_time')

Best of 3 trials with 10 function calls per trial:
Function `convert` ran in average of 0.679 seconds



In [28]:
@timeit(repeat=3, number=100)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],
                          format="%d%m%y %H:%M")

df["date_time"] = convert_with_format(df, "date_time")

Best of 3 trials with 100 function calls per trial:
Function `convert_with_format` ran in average of 0.005 seconds

